In [2]:
# export model to onnx format without weights and biases

import sys
sys.path.insert(0, '../src') # to be able to import functions from src

import torch
from unet import *

# network hyperparameters
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
in_channels=3 # rgb

m = ContextUnet(in_channels, n_feat=n_feat, n_cfeat=n_cfeat, height=height)
samples = torch.randn(9, 3, height, height) 
t = torch.tensor([1 / 500])[:, None, None, None]
torch.onnx.export(m,         # model being run 
         (samples, t),       # model input (or a tuple for multiple inputs) 
         "mymodel.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['samples', 'timestamp'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
) 

import onnx
onnx_model = onnx.load("mymodel.onnx")
onnx.checker.check_model(onnx_model)

FileNotFoundError: [Errno 2] No such file or directory: 'context_unet.onnx'